<a href="https://colab.research.google.com/github/afrina23/Single-Image-Crowd-Counting-with-MCNN/blob/master/LBF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd gdrive/My Drive/project_folder/LBF_RR

/content/gdrive/My Drive/project_folder/LBF_RR


In [0]:
!ls

README.md  src


In [0]:
!git clone https://github.com/appstud/Crowd-Counting-With-LBP.git

Cloning into 'Crowd-Counting-With-LBP'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 33 (delta 9), reused 33 (delta 9), pack-reused 0
Unpacking objects: 100% (33/33), done.


In [0]:
%cd Crowd-Counting-With-LBP

/content/gdrive/My Drive/project_folder/LBF_RR/Crowd-Counting-With-LBP


In [0]:
import scipy.io as sio
import cv2
import argparse
#import utils as utils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pdb
from skimage import feature
import numpy as np
import os
import glob
import os
import random


def get_file_id(filepath):
    return os.path.splitext(os.path.basename(filepath))[0]


def get_data_list(data_root, mode='train'):
    """
    Returns a list of images that are to be used during training, validation and testing.
    It looks into various folders depending on the mode and prepares the list.
    :param mode: selection of appropriate mode from train, validation and test.
    :return: a list of filenames of images and corresponding ground truths after random shuffling.
    """

    if mode == 'train':
        imagepath = os.path.join(data_root, 'train_data', 'images')
        gtpath = os.path.join(data_root, 'train_data', 'ground_truth')

    elif mode == 'valid':
        imagepath = os.path.join(data_root, 'valid_data', 'images')
        gtpath = os.path.join(data_root, 'valid_data', 'ground_truth')

    else:
        imagepath = os.path.join(data_root, 'test_data', 'images')
        gtpath = os.path.join(data_root, 'test_data', 'ground_truth')

    image_list = [file for file in glob.glob(os.path.join(imagepath, '*.jpg'))]
    gt_list = []

    for filepath in image_list:
        file_id = get_file_id(filepath)
        gt_file_path = os.path.join(gtpath, 'GT_' + file_id + '.mat')
        gt_list.append(gt_file_path)
    xy = list(zip(image_list, gt_list))
    random.shuffle(xy)
    s_image_list, s_gt_list = zip(*xy)

    return s_image_list, s_gt_list


class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        # store the number of points and radius
        self.numPoints = numPoints
        self.radius = radius

    def describe(self, image, eps=1e-7):
        # compute the Local Binary Pattern representation
        # of the image, and then use the LBP representation
        # to build the histogram of patterns
        lbp = feature.local_binary_pattern(image, self.numPoints,
                self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(),
                bins=np.arange(0, 60))

        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)

        # return the histogram of Local Binary Patterns
        return hist

    
def getNumberOfPointsInsideRectangle(points,rectangle={"anchor":(0,0),"width":200,"height":200}):
    ### points: a 2D array of points, each line is a sample.
    ### rectangle: a dictionary that defines a rectangle by its top left corner, width and height.
    ### returns the points inside the rectangle and their number.
    
    mask=(points[:,0]<(rectangle["anchor"][1]+rectangle["width"])) & (points[:,0]> rectangle["anchor"][1])
    mask=(mask) & (points[:,1]<(rectangle["anchor"][0]+rectangle["height"])) & (points[:,1]> rectangle["anchor"][0])
    
    return np.sum(mask),points[mask,:]

def drawPointsOnImage(image,points):
    ###image: RGB image to draw points on
    ###points: the points to be drawn
    ###returns: the image after the drawing operation
    
    for x,y in list(points):
        cv2.circle(image, (np.int(x),np.int(y)), 1, (0, 255, 0), -1)
    return image

def extractDataForOneImage(image,points,lbp,numberOfRectanglePerRow,numberOfRectanglePerColumn,showImg=0):
    
    
    ###Convert to grayscale if the image is not in grayscale already
    height,width,numChannels=image.shape
    if(numChannels==3):
        grayImage=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        grayImage=rawImage
    nbPointInImage=[]
    widthOfPatch=int(width/float(numberOfRectanglePerRow))
    heightOfPatch=int(height/float(numberOfRectanglePerColumn))
    allHistInImage=[]
    
    for i in range(numberOfRectanglePerColumn):
        for j in range(numberOfRectanglePerRow):
            ###Get the points inside a patch and their numbers
            nbPoint,headPointsInPatch=getNumberOfPointsInsideRectangle(points,rectangle={"anchor":(i*heightOfPatch,j*widthOfPatch),"width":widthOfPatch,"height":heightOfPatch})
            
            ###Add it to a vector that contains the number of points in a each patch
            nbPointInImage.append(nbPoint)

            ###Draw the boundaries of the current patch on the image
            cv2.rectangle(image,(j*widthOfPatch,i*heightOfPatch),(j*widthOfPatch+widthOfPatch,i*heightOfPatch+heightOfPatch),(0,0,255),3)
            
            ###Draw the points available in the current patch on the image
            rawImage=drawPointsOnImage(image,headPointsInPatch)

            ###Show the image
            if (showImg):
                cv2.imshow("image",rawImage)
                k=cv2.waitKey(1)

            ###Calculate the LBP uniform histogram descriptor for the current patch
            hist=lbp.describe(grayImage[i*heightOfPatch:(i+1)*heightOfPatch,j*widthOfPatch:(j+1)*widthOfPatch], eps=1e-7)

            ###Append current 59D descriptor to descriptors of previous patches in a list
            allHistInImage.append(hist)
            
    ###Concatenate all descriptors of all patches into one descriptor vector  
    allHistInImage=np.concatenate(allHistInImage, axis=0)

    return nbPointInImage,allHistInImage


data_root='/content/gdrive/My Drive/project_folder/LBF_RR/data/ShanghaiTech/part_A_final'
numberOfRectanglePerRow=16
numberOfRectanglePerColumn=12
showImg=0

def extractDescriptorsAndGroundTruthMatrix(mode="train"):
    #Get the list of images.
    images_list, gts_list = get_data_list(data_root, mode=mode)
    allPoints=None
    allHist=None
    lbp=LocalBinaryPatterns(numPoints=8, radius=1)
    for img_idx in range(len(images_list)):
        
        # Load the image and ground truth
        rawImage = np.asarray(cv2.imread(images_list[img_idx]), dtype=np.uint8)
        matFile=sio.loadmat(gts_list[img_idx])
        if img_idx%10 == 0:
            print("Image processed : "+str(img_idx)+"/"+str(len(images_list)))
        headPoints = matFile['image_info'][0][0][0][0][0]
            
        nbPointInImage,allHistInImage=extractDataForOneImage(rawImage,headPoints,lbp,numberOfRectanglePerRow,numberOfRectanglePerColumn,showImg)
        
        ###Concatenate all descriptors into one matrix and all vectors of number of points into one matrix   
        if(allPoints is None):
            allPoints=np.array(nbPointInImage)
        else:
            allPoints=np.vstack((allPoints,nbPointInImage))

        if(allHist is None):
            allHist=allHistInImage

        else:
            allHist=np.vstack((allHist,allHistInImage))
        
    ###Save the extracted descriptors matrix and ground truth matrix
    sio.savemat(os.path.join(data_root,mode+".mat"),{"lbp_descriptors":allHist,"labels":allPoints})
      
    print(mode+ "set descriptors shape:" +str(allHist.shape))
    print(mode+ "set ground truth shape:" +str(allPoints.shape))

def main():
    ###Extract descriptors and ground truth for the train set
    extractDescriptorsAndGroundTruthMatrix(mode="train")

    ###Extract descriptors and ground truth for the test set
    extractDescriptorsAndGroundTruthMatrix(mode="test")
    



if __name__ == "__main__":
    
    #parser = argparse.ArgumentParser()
    #parser.add_argument('--data_root', default='/content/gdrive/My Drive/project_folder/LBF_RR/data/ShanghaiTech/part_A_final', type=str)
    #parser.add_argument('--numberOfRectanglePerRow', default=16, type=int)
    #parser.add_argument('--numberOfRectanglePerColumn', default=12, type=int)
    #parser.add_argument('--showImg', default=0, type=int)

    #args = parser.parse_args()

    main()


Image processed : 0/300
Image processed : 10/300
Image processed : 20/300
Image processed : 30/300
Image processed : 40/300
Image processed : 50/300
Image processed : 60/300
Image processed : 70/300
Image processed : 80/300
Image processed : 90/300
Image processed : 100/300
Image processed : 110/300
Image processed : 120/300
Image processed : 130/300
Image processed : 140/300
Image processed : 150/300
Image processed : 160/300
Image processed : 170/300
Image processed : 180/300
Image processed : 190/300
Image processed : 200/300
Image processed : 210/300
Image processed : 220/300
Image processed : 230/300
Image processed : 240/300
Image processed : 250/300
Image processed : 260/300
Image processed : 270/300
Image processed : 280/300
Image processed : 290/300
trainset descriptors shape:(300, 11328)
trainset ground truth shape:(300, 192)
Image processed : 0/182
Image processed : 10/182
Image processed : 20/182
Image processed : 30/182
Image processed : 40/182
Image processed : 50/182
Ima

In [0]:
%tb

train the model below

In [0]:
import scipy.io as sio
import cv2
import argparse
#import utils as utils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pdb
from skimage import feature
import numpy as np
import os
import time 
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import make_scorer

def computeError(y, y_pred):
    return np.sqrt(np.mean((np.sum(y_pred,1)-np.sum(y,1))**2))
def computeMAE(y, y_pred):
    return np.mean((np.sum(y_pred,1)-np.sum(y,1)))

    
data_root='/content/gdrive/My Drive/project_folder/LBF_RR/data/ShanghaiTech/part_A_final'
def main():
    

    ###Read descriptors and ground truth for train set
    matFile=sio.loadmat(os.path.join(data_root,"train"))
    descriptorsTrain=matFile["lbp_descriptors"]
    labelsTrain=matFile["labels"]

    ###Read descriptors and ground truth for test set
    matFile2=sio.loadmat(os.path.join(data_root,"test"))
    descriptorsTest=matFile2["lbp_descriptors"]
    labelsTest=matFile2["labels"]
    
    print("descriptors train shape:"+ str(descriptorsTrain.shape))
    print("labels train shape:"+ str(labelsTrain.shape)+"\n")
    print("descriptors test shape:"+ str(descriptorsTest.shape))
    print("labels test shape:"+ str(labelsTest.shape)+"\n")
    
    Grid_Dict = {"alpha": [1e-13,1e-5,1e-4,1e-3,1e-2],"gamma": np.logspace(-3, 2, 10)}
    krr_Tuned = GridSearchCV(KernelRidge(kernel='rbf'), cv=4 ,param_grid=Grid_Dict, scoring=make_scorer(computeError,greater_is_better=False),refit=True)
    krr_Tuned.fit(descriptorsTrain, labelsTrain)

    
    predictionsTrain=krr_Tuned.predict(descriptorsTrain)
    MSE_train=computeError(labelsTrain, predictionsTrain)
    MAE_train=computeMAE(labelsTrain, predictionsTrain)


    ########testing###########  
    predictionsTest=krr_Tuned.predict(descriptorsTest)
    MSE_test=computeError(labelsTest, predictionsTest)
    MAE_test=computeMAE(labelsTest, predictionsTest)
    

    print("  minTestMSE:"+str( MSE_test),"  minTestMAE:"+str( MAE_test),  " minTrainMSE:"+ str(MSE_train)," minTrainMAE:"+ str(MAE_train),
          "Best alpha:"+ str(krr_Tuned.best_params_['alpha']), "Best rbf param:"+ str(krr_Tuned.best_params_['gamma']))

    
        

if __name__ == "__main__":
    
    #parser = argparse.ArgumentParser()
    #parser.add_argument('--data_root', default='/content/gdrive/My Drive/project_folder/LBF_RR/data/ShanghaiTech/part_A_final', type=str)
    #args = parser.parse_args()

    main()
    


descriptors train shape:(300, 11328)
labels train shape:(300, 192)

descriptors test shape:(182, 11328)
labels test shape:(182, 192)

  minTestMSE:434.6370654022479   minTestMAE:111.07478957206551  minTrainMSE:99.17123841283417  minTrainMAE:0.0237269178540555 Best alpha:0.01 Best rbf param:0.046415888336127795
